In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import os
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import gc
import datetime
import re
from joblib import dump, load
from sklearn.preprocessing import Imputer
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from datetime import datetime as dt

Get names of features in model.  Will have to remove extras and add missing

In [4]:
#features_in_model = pd.read_csv("./data/processed/train_processed.csv", nrows=1,index_col='TransactionID').columns

In [2]:
features_in_model = list(pd.read_pickle("./data/processed/train_processed_all_features_featurenames.pkl")['name'])

In [3]:
print(os.listdir("./data/raw"))
#train_transaction = pd.read_csv('./input/train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('./data/raw/test_transaction.csv', index_col='TransactionID')
#train_identity = pd.read_csv('./input/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('./data/raw/test_identity.csv', index_col='TransactionID')
print('dimensions:\n')
#print('train_transaction: ' + str(train_transaction.shape))
print('test_transaction: ' + str(test_transaction.shape))
#print('train_identity: ' + str(train_identity.shape))
print('test_identity: ' + str(test_identity.shape))
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')
#test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')
del test_transaction
#del test_transaction 
del test_identity 
#del test_identity
gc.collect()
print('test: ' + str(test.shape))

['.gitkeep', 'sample_submission.csv', 'test_identity.csv', 'test_transaction.csv', 'train_identity.csv', 'train_transaction.csv']
dimensions:

test_transaction: (506691, 392)
test_identity: (141907, 40)
test: (506691, 432)


In [4]:
START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
test['TransactionDT'] = test['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
test['time_year'] = test['TransactionDT'].dt.year
test['time_month'] = test['TransactionDT'].dt.month
test['time_dow'] = test['TransactionDT'].dt.dayofweek
test['time_hour'] = test['TransactionDT'].dt.hour
test['time_day'] = test['TransactionDT'].dt.day

In [5]:
test = test.drop(columns="TransactionDT")
test = test.drop(columns="time_year")
test = test.drop(columns="time_month")

In [6]:
id_30_regex={'(?i)^.*Android.*$':'Android',
               '(?i)^.*iOS.*$':'iOS',
               '(?i)^.*Windows.*$':'Windows',
               '(?i)^.*Mac\sOS.*$':'Mac'
              }
test['id_30']=test['id_30'].replace(regex=id_30_regex)

id_31_regex={'(?i)^.*samsung.*$':'samsung',
               '(?i)^.*safari.*$':'safari',
               '(?i)^.*chrome.*$':'chrome',
               '(?i)^.*firefox.*$':'firefox',
               '(?i)^.*opera.*$':'opera',
               '(?i)^.*edge.*$':'edge',
               '(?i)^.*ie\s\d{1,2}.*$':'ie',
              '(?i)^.*android.*$':'android',
              '(?i)^.*google.*$':'google',
              }

test['id_31']=test['id_31'].replace(regex=id_31_regex)

test['DeviceInfo'].value_counts()
device_regex={'^.*SM.*$|^.*(?i)SAMSUNG.*$':'Samsung',
             '(?i)^.*HUAWEI.*$|(?i)^hi\d.*$':'Huawei',
             '(?i)^.*TRIDENT.*$':'Trident',
             '(?i)rv\:.*$':'Firefox',
             '(?i)Moto.*$':'Moto',
              '(?i)Lg.*$':'Lg',
              '(?i)Linux.*$':'Linux',
             '(?i)HTC.*$':'HTC',
             '(?i)Hisense.*$':'Hisense',
             '(?i)Blade.*$':'Blade',
             '(?i)^.*XT\d+.*$':'Motorola',
             '(?i)^F\d{4}\sBuild.*$':'Sony',
             '(?i)^Lenovo.*$':'Lenovo',
             '(?i)^Redmi.*$':'Xiaomi',
             '(?i)^KFFOWI.*$':'Amazon',
             '(?i)^Pixel.*$|(?i)^Nexus.*$':'Google',
              '(?i)^Ilium.*$':'Lanix',
              '(?i)^Windows.*$':'Windows',
              '(?i)^\d{4}A.*$':'Alcatel'}
test['DeviceInfo']= test['DeviceInfo'].replace(regex=device_regex)

In [7]:
categorical_group_regex_transaction = 'ProductCD|card[1-6]|addr\d|\w_emaildomain|M[1-9]|time_|Device\w+|id_12|id_13|id_14|id_15|id_16|id_17|id_18|id_19|id_20|id_21|id_22|id_23|id_24|id_25|id_26|id_27|id_28|id_29|id_30|id_31|id_32|id_33|id_34|id_35|id_36|id_37|id_38'
categorical = []
numerical = []
for i in test:
    if re.match(categorical_group_regex_transaction,i):
        categorical.append(i)
    else: numerical.append(i)
#categorical.remove('isFraud')
#numerical.append('isFraud')

In [8]:
categorical_group_regex_transaction = 'ProductCD|card[1-6]|addr\d|\w_emaildomain|M[1-9]|time_|Device\w+|id_12|id_13|id_14|id_15|id_16|id_17|id_18|id_19|id_20|id_21|id_22|id_23|id_24|id_25|id_26|id_27|id_28|id_29|id_30|id_31|id_32|id_33|id_34|id_35|id_36|id_37|id_38'
for i in test:
    if re.match(categorical_group_regex_transaction,i):
        test[i].fillna("NAN", inplace=True)

In [9]:
categorical_group_regex_transaction = 'ProductCD|card[1-6]|addr\d|\w_emaildomain|M[1-9]|time_|Device\w+|id_12|id_13|id_14|id_15|id_16|id_17|id_18|id_19|id_20|id_21|id_22|id_23|id_24|id_25|id_26|id_27|id_28|id_29|id_30|id_31|id_32|id_33|id_34|id_35|id_36|id_37|id_38'
for i in test:
    if re.match(categorical_group_regex_transaction,i) and test[i].dtype!='O':
        test[i] = test[i].astype(str)

In [10]:
fill_NaN = Imputer(missing_values=np.nan, strategy='mean', axis=1)
test[numerical] = fill_NaN.fit_transform(test[numerical])

C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [11]:
test[numerical] = StandardScaler().fit_transform(test[numerical])

In [13]:
print(categorical)
print('\n\n')
print(numerical)

['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'time_dow', 'time_hour', 'time_day']



['TransactionAmt', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', '

reduce number of V columns with PCA:

In [12]:
print(test.shape)
Vs=[]
for i in test:
    if re.match('^V.*',i):
        Vs.append(i)
pca = PCA(n_components=30)
PCA_V = pd.DataFrame(pca.fit_transform(test[Vs]),index=test.index).add_prefix('PCA_V_')
test = test.drop(Vs, axis=1)
test = test.merge(PCA_V,left_index=True,right_index=True)
print(test.shape)

(506691, 434)
(506691, 125)


In [13]:
for i in Vs:
    numerical.remove(i)
for i in test:
    if re.match('PCA_V.*',i):
        numerical.append(i) 

In [14]:
def assign_low_freq_values_as_other(df,colname,n):
    #df = df[colname]
    topn = df[colname].value_counts(dropna=False).head(n)
    if topn.shape[0] < n:
        return df
    uniques = df[colname].unique()
    others = []
    for i in uniques:
        if i not in topn:
            others.append(i)
    for i in others:
        df[colname].replace(i,'other',inplace=True)
    return df
print(test[categorical].nunique().sum())
for i in test[categorical]:
    assign_low_freq_values_as_other(test,i,50)
print(test[categorical].nunique().sum())
#assign_low_freq_values_as_other(train_transaction,'addr2',30)

17830
1109


In [15]:
test_numerical=test[numerical]
test_categorical=test[categorical]
del test
gc.collect()

62

In [16]:
test_categorical_one_hot = pd.get_dummies(test_categorical)
del test_categorical
gc.collect()
test_categorical_one_hot.shape

(506691, 1109)

In [17]:
gc.collect()
#features = pd.concat([test_numerical, test_categorical_one_hot], axis=1)
features = pd.merge(test_numerical, test_categorical_one_hot,left_index=True, right_index=True)
del test_numerical
del test_categorical_one_hot
gc.collect()

14

In [18]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist
features,NAs = reduce_mem_usage(features)

Memory usage of properties dataframe is : 841.9541463851929  MB
******************************
Column:  TransactionAmt
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  dist1
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  dist2
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  C1
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  C2
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  C3
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  C4
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  C5
dtype before:  float64
dtype after:  float

dtype after:  float32
******************************
******************************
Column:  PCA_V_29
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  ProductCD_C
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  ProductCD_H
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  ProductCD_R
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  ProductCD_S
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  ProductCD_W
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card1_10023
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card1_10057
dtype before:  uint8
dtype

dtype after:  uint8
******************************
******************************
Column:  card2_298.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card2_310.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card2_321.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card2_327.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card2_360.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card2_361.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card2_369.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card2_375.0
dtype before:  uint8
dtype af

******************************
******************************
Column:  card3_198.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card3_199.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card3_200.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card3_203.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card3_204.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card3_210.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card3_212.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card3_213.0
dtype before:  uint8
dtype after:  uint8
********

******************************
******************************
Column:  card6_credit
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  card6_debit
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  addr1_110.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  addr1_122.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  addr1_123.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  addr1_126.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  addr1_143.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  addr1_158.0
dtype before:  uint8
dtype after:  uint8
*******

...

Column:  id_24_16.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_24_18.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_24_19.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_24_20.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_24_21.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_24_22.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_24_24.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_24_25.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_24_2

dtype after:  uint8
******************************
******************************
Column:  id_26_134.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_26_135.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_26_137.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_26_138.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_26_141.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_26_142.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_26_143.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_26_144.0
dtype before:  uint8
dtype af

******************************
******************************
Column:  id_31_samsung
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_31_uc
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_32_8.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_32_16.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_32_24.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_32_32.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_32_48.0
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_32_NAN
dtype before:  uint8
dtype after:  uint8
*****************

******************************
******************************
Column:  id_38_NAN
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  id_38_T
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  DeviceType_NAN
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  DeviceType_desktop
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  DeviceType_mobile
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  DeviceInfo_5025G Build/LMY47I
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  DeviceInfo_5054S Build/LMY47V
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  DeviceInfo_8050

dtype after:  uint8
******************************
******************************
Column:  time_hour_12
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  time_hour_13
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  time_hour_14
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  time_hour_15
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  time_hour_16
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  time_hour_17
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  time_hour_18
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  time_hour_19
dtype before:  uint8


In [26]:
len(features_in_model)

18398

In [ ]:
for i in features:
    if i not in features_in_model:
        #features.pop(i)
        print(removing +'i '+'column')
for i in features_in_model:
    if i not in features.columns:
        #features[i]=0
        print(adding +'i ' + 'column')
        #print(i)
features.pop('isFraud')
print('removed isFraud column')

In [ ]:
features.isna().sum().sum()

**Load and execute saved model on Test data**

In [42]:
clf = load('./data/interim/04-09-2019_14-45-04_random_foest_base.joblib')

In [43]:
y_pred=clf.predict_proba(features)

ValueError: Number of features of the model must match the input. Model n_features is 1220 and input n_features is 1250 

In [27]:
y_pred[:,1]

array([0.25293345, 0.24940112, 0.22475564, ..., 0.26289649, 0.23893397,
       0.3010366 ])

In [25]:
features['isFraud']=y_pred[:,1]

In [44]:
for i in features:
    print(i)

TransactionAmt
dist1
dist2
C1
C2
C3
C4
C5
C6
C7
C8
C9
C10
C11
C12
C13
C14
D1
D2
D3
D4
D5
D6
D7
D8
D9
D10
D11
D12
D13
D14
D15
id_01
id_02
id_03
id_04
id_05
id_06
id_07
id_08
id_09
id_10
id_11
PCA_V_0
PCA_V_1
PCA_V_2
PCA_V_3
PCA_V_4
PCA_V_5
PCA_V_6
PCA_V_7
PCA_V_8
PCA_V_9
PCA_V_10
PCA_V_11
PCA_V_12
PCA_V_13
PCA_V_14
PCA_V_15
PCA_V_16
PCA_V_17
PCA_V_18
PCA_V_19
PCA_V_20
PCA_V_21
PCA_V_22
PCA_V_23
PCA_V_24
PCA_V_25
PCA_V_26
PCA_V_27
PCA_V_28
PCA_V_29
ProductCD_C
ProductCD_H
ProductCD_R
ProductCD_S
ProductCD_W
card1_10023
card1_10057
card1_10112
card1_10486
card1_10616
card1_11207
card1_12501
card1_12544
card1_12577
card1_12695
card1_12839
card1_12932
card1_13832
card1_15063
card1_15066
card1_15497
card1_15885
card1_16075
card1_16132
card1_16136
card1_16560
card1_16659
card1_17188
card1_18132
card1_2616
card1_2803
card1_2884
card1_3154
card1_4461
card1_4806
card1_5033
card1_5812
card1_6019
card1_7207
card1_7508
card1_7585
card1_7664
card1_7676
card1_7815
card1_7826
card1_7861
card1_7919
car

In [28]:
datetimestring=dt.now().strftime("%d-%m-%Y_%H-%M-%S")
os.mkdir('./data/predictions/'+datetimestring)
features['TransactionID']=list(features.index)
features[['TransactionID','isFraud']].to_csv('./data/predictions/'+datetimestring+'/prediction_results.csv'+datetimestring,index = False)
dump(clf, './data/predictions/'+datetimestring+'/randomforest_model_new.joblib')

['./data/predictions/04-09-2019_10-36-01/randomforest_model_new.joblib']

In [40]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' By Fred Cirera, after https://stackoverflow.com/a/1094933/1870254'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name,value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name,sizeof_fmt(size)))

                      features:   7.6GiB
                         PCA_V: 139.8MiB
                            _3:   1.2MiB
                           _28:   1.2MiB
                           _31:   1.2MiB
                           ___: 161.8KiB
             features_in_model: 161.8KiB
                           _34: 161.8KiB
                            _5:  84.5KiB
                            Vs:   2.8KiB
